# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries

import numpy as np
import pandas as pd

from sqlalchemy import create_engine
import sqlite3

import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.base import BaseEstimator, TransformerMixin


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql("SELECT * FROM MSG_CATEG", engine)
X = df['message']
Y = df.iloc[:,4:]

In [3]:
display(df.head())
display(X.head())
print(Y.head())

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

   related  request  offer  aid_related  medical_help  medical_products  \
0        1        0      0            0             0                 0   
1        1        0      0            1             0                 0   
2        1        0      0            0             0                 0   
3        1        1      0            1             0                 1   
4        1        0      0            0             0                 0   

   search_and_rescue  security  military  child_alone      ...        \
0                  0         0         0            0      ...         
1                  0         0         0            0      ...         
2                  0         0         0            0      ...         
3                  0         0         0            0      ...         
4                  0         0         0            0      ...         

   aid_centers  other_infrastructure  weather_related  floods  storm  fire  \
0            0                     0  

### 2. Write a tokenization function to process your text data

In [4]:
# below was taken from the udacity course- Machine Learning Workflow exercise / ml_workflow
def tokenize(message):
    # we split into words
    words = word_tokenize(message)
    # we lemmatize (we reduce to root form)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for w in words:
        clean_tok = lemmatizer.lemmatize(w)
        # changing all to lower capital leters
        clean_tok_lower = clean_tok.lower()
         # trimming the blank spaces before and after
        clean_tok_strip = clean_tok_lower.strip()
         # adding elements to the list
        clean_tokens.append(clean_tok_strip)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
# Chose Random Forest Classifier as this is what we also used in the class- with the exercises

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(DecisionTreeClassifier(random_state=0)))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
# splitting data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, Y)
# training the pipeline
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...action_leaf=0.0, presort=False, random_state=0,
            splitter='best'),
           n_jobs=1))])

In [7]:
print(y_train.shape)
print(X_train.shape)
print(y_test.shape)
print(X_test.shape)

(19662, 36)
(19662,)
(6554, 36)
(6554,)


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
# predict on test data
y_pred=pipeline.predict(X_test)

In [9]:
# personal check to find the accuracy and f1 score for all 36 categories

nr_categ = len(Y.columns.tolist())

for i in range(nr_categ):
    class_rep=classification_report(y_test.iloc[:,i], y_pred[:,i])
    print(class_rep)

f1=[]
accuracy = (y_pred == y_test).mean().tolist()
for i in range(nr_categ):
    f1.append(f1_score(y_test.iloc[:,i], y_pred[:,i], average='weighted'))

print('On average, the accuracy for all 36 categories is: {}'.format((y_pred == y_test).mean().mean()))
print('On average, the F1 score for all our categories is {}'. format(sum(f1)/len(f1)))

for i in range(nr_categ):
     print('For category {}: Accuracy = {} , F1 Score = {}'.format(y_test.columns[i], accuracy[i], f1[i]))

             precision    recall  f1-score   support

          0       0.47      0.42      0.44      1566
          1       0.82      0.85      0.83      4940
          2       0.34      0.21      0.26        48

avg / total       0.73      0.74      0.74      6554

             precision    recall  f1-score   support

          0       0.91      0.92      0.92      5448
          1       0.59      0.57      0.58      1106

avg / total       0.86      0.86      0.86      6554

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6526
          1       0.00      0.00      0.00        28

avg / total       0.99      0.99      0.99      6554

             precision    recall  f1-score   support

          0       0.74      0.73      0.74      3873
          1       0.62      0.63      0.63      2681

avg / total       0.69      0.69      0.69      6554

             precision    recall  f1-score   support

          0       0.95      0.95 

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
# below taken from the exercise grid_search in Machine Learning Pipelines chapter
parameters = {
        'clf__estimator__criterion': ['gini', 'entropy'],
        'clf__estimator__max_features': ['sqrt', 'log2'],
        'clf__estimator__min_samples_split':[20]#,
        # best 'clf__estimator__max_depth' : [20] f1 0.93 
#'clf__estimator__max_depth' : [2, 5, 7, 12]         
    }

grid_search = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
# training the model with the new parameters

grid_search.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...action_leaf=0.0, presort=False, random_state=0,
            splitter='best'),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__criterion': ['gini', 'entropy'], 'clf__estimator__max_features': ['sqrt', 'log2'], 'clf__estimator__min_samples_split': [20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [12]:
print(grid_search.best_params_)

{'clf__estimator__criterion': 'entropy', 'clf__estimator__max_features': 'log2', 'clf__estimator__min_samples_split': 20}


In [13]:
# creating a new model
grid_search_model = grid_search.best_estimator_
# predicting on the test data
y_pred_new = grid_search_model.predict(X_test)

In [14]:
# personal check to better see the accuracy and f1 score for all 36 categories

f1=[]
accuracy = (y_pred_new == y_test).mean().tolist()
for i in range(len(y_test.columns)):
    f1.append(f1_score(y_test.iloc[:,i], y_pred_new[:,i], average='weighted'))

print('On average, the accuracy for all 36 categories is: {}'.format((y_pred_new == y_test).mean().mean()))
print('On average, the F1 score for all our categories is {}'. format(sum(f1)/len(f1)))

for i in range(len(y_test.columns)):
     print('For category {}: Accuracy = {} , F1 Score = {}'.format(y_test.columns[i], accuracy[i], f1[i]))

On average, the accuracy for all 36 categories is: 0.919993727325128
On average, the F1 score for all our categories is 0.9115395012341475
For category related: Accuracy = 0.7149832163564236 , F1 Score = 0.7070950013329851
For category request: Accuracy = 0.8310955141898078 , F1 Score = 0.8269951380483643
For category offer: Accuracy = 0.9952700640830028 , F1 Score = 0.9936259132547068
For category aid_related: Accuracy = 0.6473909063167531 , F1 Score = 0.6461059804383529
For category medical_help: Accuracy = 0.9047909673481843 , F1 Score = 0.8918867052127468
For category medical_products: Accuracy = 0.9409520903265182 , F1 Score = 0.9317206525162305
For category search_and_rescue: Accuracy = 0.9642966127555691 , F1 Score = 0.9568511469099779
For category security: Accuracy = 0.9760451632590784 , F1 Score = 0.9685203364761615
For category military: Accuracy = 0.9610924626182484 , F1 Score = 0.9538562913386774
For category child_alone: Accuracy = 1.0 , F1 Score = 1.0
For category water:

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
#  trying a random forest classifier

from sklearn.tree import DecisionTreeClassifier

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

# splitting data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)
y_pred=pipeline.predict(X_test)


In [17]:
f1=[]
accuracy = (y_pred == y_test).mean().tolist()
for i in range(nr_categ):
    f1.append(f1_score(y_test.iloc[:,i], y_pred[:,i], average='weighted'))
print('On average, the accuracy for all 36 categories is: {}'.format((y_pred == y_test).mean().mean()))
print('On average, the F1 score for all our categories is {}'. format(sum(f1)/len(f1)))
for i in range(nr_categ):
     print('For category {}: Accuracy = {} , F1 Score = {}'.format(y_test.columns[i], accuracy[i], f1[i]))

On average, the accuracy for all 36 categories is: 0.9422574848269082
On average, the F1 score for all our categories is 0.9277488313398611
For category related: Accuracy = 0.790051876716509 , F1 Score = 0.7660756197873474
For category request: Accuracy = 0.8802258162953921 , F1 Score = 0.8620354972155897
For category offer: Accuracy = 0.9967958498626793 , F1 Score = 0.9951963455571
For category aid_related: Accuracy = 0.7326823314006713 , F1 Score = 0.7210165371112199
For category medical_help: Accuracy = 0.9212694537686908 , F1 Score = 0.8907552418022251
For category medical_products: Accuracy = 0.9491913335367714 , F1 Score = 0.9278335050036332
For category search_and_rescue: Accuracy = 0.9734513274336283 , F1 Score = 0.961724334507846
For category security: Accuracy = 0.9826060421116876 , F1 Score = 0.9741366041621333
For category military: Accuracy = 0.9659749771132133 , F1 Score = 0.9523966701365174
For category child_alone: Accuracy = 1.0 , F1 Score = 1.0
For category water: Acc

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [ ]:
# below taken form https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/
import pickle
pickle.dump(grid_search_model, open('model.pkl', 'wb'))
print('DONE')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.